In [17]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from datetime import datetime, timedelta,date
import warnings
warnings.filterwarnings('ignore')

In [49]:
#Useful functions
def calculate_sum(state_df,state_name,column):
    
    '''selects the total number from state specified in the argument
    For e.g: If we want total from Alabama state of Confirmed case column it returns a number'''
    
    total = state_df[state_df['Province_State']==state_name][column].values[0]
    return total

def calculate_state_to_county(county_df,state_df,state_name,first_column,second_column):
    new_df = county_df[county_df['State']==state_name]
    first_col_total = calculate_sum(state_df,state_name,first_column)
    second_col_total = calculate_sum(state_df,state_name,second_column)

    return (round((new_df[first_column]/first_col_total)*second_col_total)).tolist()
def drop_and_melt(us_confirmed_raw,us_deaths_raw):
    #Drop uncessary columns
    confirmed_df = us_confirmed_raw.drop(['UID','iso2',"iso3",'code3','Country_Region','Combined_Key'],axis=1)

    #Drop uncessary columns

    deaths_df = us_deaths_raw.drop(['UID','iso2',"iso3",'code3','Country_Region','Combined_Key'],axis=1)

    #Extract variable with dates
    dates = confirmed_df.columns[6:]

    #Melt the dataset from row into columns
    us_confirmed = pd.melt(confirmed_df,id_vars = ['Admin2','Province_State','FIPS','Lat','Long_'],value_vars=dates, var_name='Date', value_name='Confirmed')
    us_deaths = pd.melt(deaths_df,id_vars = ['Admin2','Province_State','FIPS','Lat','Long_'],value_vars=dates, var_name='Date', value_name='Deaths')
    us_confirmed['Date'] = pd.to_datetime(us_confirmed['Date'],format = '%m/%d/%y')
    us_deaths['Date'] = pd.to_datetime(us_deaths['Date'], format = '%m/%d/%y')
    return us_confirmed, us_deaths

def calculate_average_new_cases(df):
    
    yesterday = df['Date'].max().strftime('%m/%d/%y')
    days_14 = (df['Date'].max()- timedelta(days = 14)).strftime('%m/%d/%y') 
    df = df.set_index('Date')
    days_14 = df.loc[days_14:yesterday]
    days_14.sort_values(by = ['Province_State','Admin2'])
    state_county = days_14[['Province_State','Admin2']]
    state_county = state_county.drop_duplicates()
    state_county = state_county.reset_index()
    state = []
    county = []
    average  = []
    for i in range(0,len(state_county)):
        temp_state = state_county.loc[i,'Province_State']
        temp_county = state_county.loc[i,'Admin2']
        mean_average = days_14[(days_14['Province_State'] == temp_state) & (days_14['Admin2']==temp_county)]['Confirmed'].diff().mean()
        state.append(temp_state)
        county.append(temp_county)
        average.append(mean_average)
    average_df = pd.DataFrame({'state':state,'county':county,'average':average})
    return average_df

def prepare_data(us_confirmed,us_deaths,pop_mob_df_raw,us_land):
    #Filter data from recent date
    latest = us_confirmed['Date'].max().strftime('%Y-%m-%d')
    counties_confirmed = us_confirmed[us_confirmed['Date']==latest]
    counties_deaths = us_deaths[us_deaths['Date']==latest]['Deaths']

    #Concatenate two dataframes
    us_counties_df_raw = pd.concat([counties_confirmed,counties_deaths],axis=1)

    #Remove missing counties as they are from other US territories
    us_counties_df_raw = us_counties_df_raw[us_counties_df_raw['Admin2'].notnull()]

    #Remove values which are not in US states
    not_us_states = ['Puerto Rico','District of Columbia']
    us_counties_df_raw = us_counties_df_raw[~us_counties_df_raw['Province_State'].isin(not_us_states)]

    #     unused_cols = ['Country_Region','Last_Update','FIPS','UID','ISO3','USAState','TotalRecovered']
    #     us_states_df = us_states_df.drop(columns=unused_cols,axis=1)

    #Remove unknown county names
    us_counties_df_raw = us_counties_df_raw[~us_counties_df_raw['Admin2'].str.contains(r'Out .*|Unassigned',regex=True,na=False)]

    # Match county names for two columns
    pop_mob_df_raw['county'] = pop_mob_df_raw['county'].str.split(' County')
    pop_mob_df_raw.loc[:, 'County'] = pop_mob_df_raw.county.map(lambda x: x[0])

    #Drop extra column
    pop_mob_df_raw = pop_mob_df_raw.drop('county',axis=1)

    #Filter values which are not considered as county
    pop_mob_df_raw = pop_mob_df_raw[pop_mob_df_raw['state']!='District of Columbia']
    pop_mob_df_raw = pop_mob_df_raw.dropna(axis=1)
    #Create stopwords which do not match county names from JHU data
    stopwords = ['Census Area','Parish','Municipality','Parish','city and Borough','city',' and Borough','Borough','City']

    #Remove stopwords
    for stopword in stopwords:
        pop_mob_df_raw['County']  = pop_mob_df_raw['County'].str.replace(stopword,'').str.strip()

    pop_mob_df_raw = pop_mob_df_raw.sort_values(by='population2019')
    pop_mob_df_raw = pop_mob_df_raw.drop_duplicates(subset= ['state','County'],keep='last')

    us_counties_df = us_counties_df_raw.merge(pop_mob_df_raw,how="inner",left_on=["Province_State","Admin2"],right_on=["state","County"])

    #Drop duplicate columns
    cols_to_be_removed = ['Admin2','Province_State']
    us_counties_df = us_counties_df.drop(columns=cols_to_be_removed,axis=1)

    #Rename and reorder colunmns
    us_counties_df = us_counties_df.rename(columns = {'state':'State','Long_':'Long','population2019':'Population'})
    columns = ['Date','State','County','FIPS','Lat','Long','Population','Confirmed','Deaths','Recovered','Mortality_Rate']
    us_counties_df = us_counties_df.reindex(columns,axis=1)

    # Add land area to master dataframe

    #Remove whitespaces
    us_land['State'] = us_land['State'].str.strip()
    us_land = us_land.sort_values(by='Area')
    us_land = us_land.drop_duplicates(subset= ['State','County'],keep='last')
    #Merge dataset on common columns
    us_counties_df = us_counties_df.merge(us_land,how="left",on=["State","County"])
    return us_counties_df

def clean_us_states(world_us_states_df):
    #Remove unwanted values by index
    world_us_states_df = world_us_states_df.drop(world_us_states_df.index[[0,63]])

    #Extract only useful columns
    world_us_states_df = world_us_states_df[['USAState','TotalRecovered']]

    #Merge two dataframe on state names
    us_states_df = us_states_df_raw.merge(world_us_states_df,how='inner',left_on='Province_State',right_on='USAState')

    #Fill missing values
    us_states_df['Recovered'] = us_states_df['Recovered'].fillna(us_states_df['TotalRecovered'],axis=0)

    #Remove values which are not in US states
    remove_states = ['Guam','Puerto Rico','Northern Mariana Islands']
    us_states_df = us_states_df[~us_states_df['Province_State'].isin(remove_states)]
    unused_cols = ['Country_Region','Last_Update','FIPS','UID','ISO3','USAState','TotalRecovered']
    us_states_df = us_states_df.drop(columns=unused_cols,axis=1)
    return us_states_df

def create_new_features(us_states_df,us_counties_df):
    #Create list of states
    states = list(us_states_df['Province_State'])
    recovered_list = []
    test_list = []

    for state in states:
        recovered_list.append(calculate_state_to_county(us_counties_df,us_states_df,state,'Confirmed','Recovered'))
    #Convert lists of list into single list
    flattened = [values for sublist in recovered_list for values in sublist]

    #Add list values into dataframe corsspending to each row
    us_counties_df['Recovered'] = flattened

    #Create new column for mortality rates
    us_counties_df['Mortality_Rate'] = (us_counties_df['Deaths'] *100) / us_counties_df['Confirmed']

    # Convert land area square miles into square kms
    miles_to_km = 0.38610
    us_counties_df['Land Area'] = us_counties_df['Area']/miles_to_km
    us_counties_df = us_counties_df.drop(columns='Area',axis=1)

    return us_counties_df
def validate_data(us_counties_df):
    df = us_counties_df.copy()

    df['Active'] = df['Confirmed'] - df['Recovered'] - df['Deaths']
    df['Active'][df['Active']<0] = 0
    #Death rate tried with confirmed cases but values were ununsual and not relaiable
    # source: https://www.cdc.gov/csels/dsepd/ss1978/lesson3/section3.html
    df['Death_per_1000_pop'] = (df['Deaths'] /df['Population'])*1000
    df['Infection_Rate'] = (df['Active']/df['Population'])*100
    reg_url = 'https://www.indexmundi.com/facts/united-states/quick-facts/louisiana/land-area#table'
    req = Request(url=reg_url, headers=headers)
    html = urlopen(req).read()
    lous_county_df = pd.read_html(html)[0]
    lous_county_df['County'] = lous_county_df['County'].str.replace('Parish','').str.strip()
    lousiana = df[df['State']=='Louisiana']
    merged_lous = lousiana.merge(lous_county_df,on='County')
    merged_lous = merged_lous.drop('Land Area',axis=1)
    miles_to_km = 0.38610
    merged_lous['Value'] = merged_lous['Value']/miles_to_km
    merged_lous = merged_lous.rename(columns={'Value':'Land Area'})
    df = df[df['State']!='Louisiana']
    df = pd.concat([df,merged_lous],axis=0)
    #change the merge code to this one later
    df.loc[df['County']=='Kusilvak','Land Area'] = 50953
    df.loc[df['County']=='Oglala Lakota','Land Area']  = 5431
    df.loc[df['County']=='Denali','Land Area'] = 33092
    df.loc[df['County']=='Skagway','Land Area'] = 28267
    df.loc[df['County']=='Wrangell','Land Area'] =9004
    df.loc[df['County']=='Yakutat','Land Area'] = 24509
    df.loc[df['County']=='Broomfield','Land Area'] = 86.89
    df.loc[df['County']=='Fairfax','Land Area'] = 1052
    df.loc[df['County']=='Fairfax','Population'] = 1010000
    df['Pop_density'] = df['Population']/df['Land Area']
    df['Active_per_1000_pop'] = (df['Active']/df['Population'])*1000
    df['Recovered_per_1000_pop'] = (df['Recovered']/df['Population'])*1000
    return df
def merge_average_df(df,average_df):
    final_df = df.merge(average_df,how='left',left_on=['State','County'],right_on=['state','county'])
    return final_df

def drop_unsed_columns(df):
    df = df.drop(columns=['state','county'])
    return df

In [50]:
# US Sate level data from JHU latest retrieval
us_states_df_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/08-20-2020.csv')

# Us confirmed Cases County level from JHU
us_confirmed_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# Us deaths County level from JHU
us_deaths_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

# From Worldometer
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
reg_url = 'https://www.worldometers.info/coronavirus/country/us/'
req = Request(url=reg_url, headers=headers)
html = urlopen(req).read()
world_us_states_df = pd.read_html(html)[0]

# From google developers contaning state names
us_states = pd.read_html('https://developers.google.com/public-data/docs/canonical/states_csv')[0]

#From google mobility and census 
pop_mob_df_raw = pd.read_csv('pop_mob_us.csv',index_col = 0)
#From census.gov
us_land = pd.read_csv('us_area.csv')

In [51]:
us_confirmed, us_deaths = drop_and_melt(us_confirmed_raw,us_deaths_raw)
average_df = calculate_average_new_cases(us_confirmed)
us_counties_df = prepare_data(us_confirmed,us_deaths,pop_mob_df_raw,us_land)
us_states_df = clean_us_states(world_us_states_df)
us_counties_df = create_new_features(us_states_df,us_counties_df)
cleaned_df = validate_data(us_counties_df)
df = merge_average_df(cleaned_df,average_df)
final_df = drop_unsed_columns(df)